# Project 1 – Decision Trees and Random Forests

In [30]:
# Reload all modules without having to restart the kernel
# Useful for development if you have edited any of the external code files.
%load_ext autoreload
%autoreload 2

# Imports
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
# ... add more imports as needed
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score

# My implementations
from decision_tree import DecisionTree
from random_forest import RandomForest

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Dataset

Do data loading, exploration and preprocessing as you see fit.

Here is some code to load the dataset to get you started.

In [31]:
data = np.genfromtxt("wine_dataset_small.csv", delimiter=",", dtype=float, names=True)

feature_names = list(data.dtype.names[:-1])
target_name = data.dtype.names[-1]

X = np.array([data[feature] for feature in feature_names]).T
y = data[target_name].astype(int)

print(f"Feature columns names: {feature_names}")
print(f"Target column name: {target_name}")
print(f"X shape: {X.shape}")
print(f"y shape: {y.shape}")

Feature columns names: ['citric_acid', 'residual_sugar', 'pH', 'sulphates', 'alcohol']
Target column name: type
X shape: (500, 5)
y shape: (500,)


Checking if the dataset is imbalanced:

In [32]:
print(np.count_nonzero(y)/len(y))
count=0
for n in y:
    if n==0:
        count+=1
print(count/len(y))

0.51
0.49


Splitting my data 

In [33]:
seed = 0 
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.2, random_state=seed)

# 3.1 Model selection 

In [34]:
# Model selection for my own implementation 
from sklearn.model_selection import KFold

hyper_parameter_grid = {
    "n_estimators": [10, 25, 50, 100],
    "max_depth": [5, 7, 10, 12],
    "criterion": ["entropy", "gini"],
    "max_features": ["sqrt", "log2"]
}

In [35]:
# This method takes in a hyperparamer_grid, training data X with target values y. 
#   It also takes optional arguments for which decision tree implementation to choose. 
#   It also has the optional performance meassure.
# It uses K-fold cross validation on the dataset for each hyperparameter combination.
# Returns the highest mean validation accuracy during tuning process,
# and best hyperparameters for both decision tree and random forest.

def model_selection(hyper_parameter_grid, X, y, imp_dt=DecisionTree, imp_rf=RandomForest, p_meassure=accuracy_score):
    highest_accuracy_dt = 0
    highest_accuracy_rf = 0

    kf = KFold(n_splits=5, shuffle=True, random_state=seed)

    X_np = np.array(X) 
    y_np = np.array(y)

    for max_depth in hyper_parameter_grid["max_depth"]:
        for criterion in hyper_parameter_grid["criterion"]:
                
            cv_valid_acc_dt = []
            
            for train_index, val_index in kf.split(X_np):
                X_train_cv, X_val_cv = X_np[train_index], X_np[val_index]
                y_train_cv, y_val_cv = y_np[train_index], y_np[val_index]
                
                dt = imp_dt(max_depth=max_depth, criterion=criterion, random_state=seed)
                dt.fit(X_train_cv, y_train_cv)
                
                valid_accuracy_dt = p_meassure(y_val_cv, dt.predict(X_val_cv))
                
                cv_valid_acc_dt.append(valid_accuracy_dt)
            
            avg_valid_acc_dt = np.mean(cv_valid_acc_dt)

            if avg_valid_acc_dt > highest_accuracy_dt:
                highest_accuracy_dt = avg_valid_acc_dt
                best_parameters_dt = max_depth, criterion

            for max_features in hyper_parameter_grid["max_features"]:
                for n_estimators in hyper_parameter_grid["n_estimators"]:
                    cv_valid_acc_rf = []
                    
                    for train_index, val_index in kf.split(X_np):
                        X_train_cv, X_val_cv = X_np[train_index], X_np[val_index]
                        y_train_cv, y_val_cv = y_np[train_index], y_np[val_index]
                        
                        rf = imp_rf(n_estimators=n_estimators, max_depth=max_depth, criterion=criterion, max_features=max_features, random_state=seed)
                        rf.fit(X_train_cv, y_train_cv)

                        valid_accuracy_rf = p_meassure(y_val_cv, rf.predict(X_val_cv))
                        
                        cv_valid_acc_rf.append(valid_accuracy_rf)
                    
                    avg_valid_acc_rf = np.mean(cv_valid_acc_rf)
                    
                    if avg_valid_acc_rf > highest_accuracy_rf:
                        highest_accuracy_rf = avg_valid_acc_rf
                        best_parameters_rf = n_estimators, max_depth, criterion, max_features

    return highest_accuracy_dt, best_parameters_dt, highest_accuracy_rf, best_parameters_rf

In [36]:
highest_accuracy_dt, best_parameters_dt, highest_accuracy_rf, best_parameters_rf = model_selection(hyper_parameter_grid, X_train_val, y_train_val)
max_depth_dt, criterion_dt = best_parameters_dt
n_estimators, max_depth_rf, criterion_rf, max_features = best_parameters_rf
main_model_dt = DecisionTree(max_depth=max_depth_dt, criterion=criterion_dt, random_state=seed)
main_model_rf = RandomForest(n_estimators=n_estimators, max_depth=max_depth_rf, criterion=criterion_rf, max_features=max_features, random_state=seed)
main_model_dt.fit(X_train_val, y_train_val)
main_model_rf.fit(X_train_val, y_train_val)

print(f"The best validation accuracy for Decision Tree implementation: {highest_accuracy_dt}")
print(f"The best parameters for the Decision Tree are {best_parameters_dt}")
print(f"The best validation accuracy for Random Forest implementation: {highest_accuracy_rf}")
print(f"The best parameters for the Random Forest are {best_parameters_rf}")

The best validation accuracy for Decision Tree implementation: 0.805
The best parameters for the Decision Tree are (5, 'gini')
The best validation accuracy for Random Forest implementation: 0.865
The best parameters for the Random Forest are (50, 7, 'entropy', 'sqrt')


# 3.2 Model evaluation and comparing to existing implementation

In [37]:
# Test accuracy score for my own implementation
print(f"The test accuracy for Decision Tree implementation: {accuracy_score(y_test, main_model_dt.predict(X_test))}")
print(f"The test accuracy for Random Forest implementation: {accuracy_score(y_test, main_model_rf.predict(X_test))}")

The test accuracy for Decision Tree implementation: 0.87
The test accuracy for Random Forest implementation: 0.9


In [38]:
# Test accuracy score on existing implementation
highest_accuracy_dt2, best_parameters_dt2, highest_accuracy_rf2, best_parameters_rf2 = model_selection(hyper_parameter_grid, X_train_val, y_train_val, imp_dt=DecisionTreeClassifier, imp_rf=RandomForestClassifier)
max_depth_dt, criterion_dt = best_parameters_dt2
n_estimators, max_depth_rf, criterion_rf, max_features = best_parameters_rf2
main_model_dt2 = DecisionTreeClassifier(max_depth=max_depth_dt, criterion=criterion_dt, random_state=seed)
main_model_rf2 = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth_rf, criterion=criterion_rf, max_features=max_features, random_state=seed)
main_model_dt2.fit(X_train_val, y_train_val)
main_model_rf2.fit(X_train_val, y_train_val)
print(f"The test accuracy for sklearns Decision Tree Classifier implementation: {accuracy_score(y_test, main_model_dt2.predict(X_test))}")
print(f"The best parameters for the sklearn Decision Tree Classifier are {best_parameters_dt2}")
print(f"The test accuracy for sklearns Random Forest Classifier implementation: {accuracy_score(y_test, main_model_rf2.predict(X_test))}")
print(f"The best parameters for the sklearn Random Forest Classifier are {best_parameters_rf2}")

The test accuracy for sklearns Decision Tree Classifier implementation: 0.82
The best parameters for the sklearn Decision Tree Classifier are (5, 'gini')
The test accuracy for sklearns Random Forest Classifier implementation: 0.91
The best parameters for the sklearn Random Forest Classifier are (100, 10, 'entropy', 'sqrt')


# 3.3 Another dataset
Loading and splitting the coffee dataset.


In [39]:
data = np.genfromtxt("coffee_data.csv", delimiter=",", dtype=float, names=True)

feature_names = list(data.dtype.names[:-1])
target_name = data.dtype.names[-1]

X2 = np.array([data[feature] for feature in feature_names]).T
y2 = data[target_name].astype(int)

X_train_val2, X_test2, y_train_val2, y_test2 = train_test_split(X2, y2, test_size=0.2, random_state=seed)

print(f"Feature columns names: {feature_names}")
print(f"Target column name: {target_name}")
print(f"X shape: {X2.shape}")
print(f"y shape: {y2.shape}")

Feature columns names: ['Aroma', 'Flavor', 'Aftertaste', 'Acidity', 'Body', 'Balance', 'Uniformity', 'Sweetness']
Target column name: CountryofOrigin
X shape: (419, 8)
y shape: (419,)


Checking how balanced the coffee dataset is

In [40]:
print(np.count_nonzero(y2)/len(y2))
count=0
for n in y2:
    if n==0:
        count+=1
print(count/len(y2))

0.5632458233890215
0.43675417661097854


Here I am doing the model selection process again, only this time with a coffee dataset.

In [41]:
highest_accuracy_dt3, best_parameters_dt3, highest_accuracy_rf3, best_parameters_rf3 = model_selection(hyper_parameter_grid, X_train_val2, y_train_val2, p_meassure=f1_score)
max_depth_dt, criterion_dt = best_parameters_dt3
n_estimators, max_depth_rf, criterion_rf, max_features = best_parameters_rf3
main_model_dt3 = DecisionTree(max_depth=max_depth_dt, criterion=criterion_dt, random_state=seed)
main_model_rf3 = RandomForest(n_estimators=n_estimators, max_depth=max_depth_rf, criterion=criterion_rf, max_features=max_features, random_state=seed)
main_model_dt3.fit(X_train_val2, y_train_val2)
main_model_rf3.fit(X_train_val2, y_train_val2)
print(f"The best validation accuracy for Decision Tree implementation: {highest_accuracy_dt3}")
print(f"The best parameters for the Decision Tree are {best_parameters_dt3}")
print(f"The best validation accuracy for Random Forest implementation: {highest_accuracy_rf3}")
print(f"The best parameters for the Random Forest are {best_parameters_rf3}")

The best validation accuracy for Decision Tree implementation: 0.7973533774738593
The best parameters for the Decision Tree are (5, 'entropy')
The best validation accuracy for Random Forest implementation: 0.8250826839136867
The best parameters for the Random Forest are (50, 10, 'gini', 'sqrt')


Printing the test results for the coffee dataset, using f1-score as performance meassure

In [42]:
print(f"The test accuracy for Decision Tree implementation: {f1_score(y_test2, main_model_dt3.predict(X_test2))}")
print(f"The test accuracy for Random Forest implementation: {f1_score(y_test2, main_model_rf3.predict(X_test2))}")

The test accuracy for Decision Tree implementation: 0.8636363636363636
The test accuracy for Random Forest implementation: 0.8666666666666667
